In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Load data

In [ ]:
df = pd.read_csv('Housing.csv')
df.head()

In [ ]:
df.info()

In [ ]:
#Encode categorical data
def encode_yes_no(value):
    return 1 if value == 'yes' else 0 if value == 'no' else value # in columns w/ yes or no we swap to 1 & 0

for col in df.select_dtypes(include=['object']).columns:
    if set(df[col].unique()) <= {'yes', 'no'}:
        df[col] = df[col].apply(encode_yes_no)

df['furnishingstatus'].replace(to_replace=['furnished', 'semi-furnished', 'unfurnished'], value=[2, 1, 0], inplace=True) #in this column we swap to 2, 1, 0

# Data preprocessing

In [ ]:
#Check for missing values
df.isnull().sum()

# Correlation Analysis

In [ ]:
#Pearson's corr
plt.figure(figsize=(12,8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.title('Pearson Feature Correlation')
plt.show()

In [ ]:
#Kendall
plt.figure(figsize=(12,8))
sns.heatmap(df.corr(method='kendall'), annot=True, cmap='coolwarm')
plt.title('Kendall Feature Correlation')
plt.show()

In [ ]:
#Spearman
plt.figure(figsize=(12,8))
sns.heatmap(df.corr(method='spearman'), annot=True, cmap='coolwarm')
plt.title('Spearman Feature Correlation')
plt.show()

In [ ]:
#Masking redundant values that do not have high correlation
corr = df.corr(method='kendall')
threshold = 0.19  # Adjust as needed

# Mask values below the threshold
mask = np.abs(corr) < threshold

plt.figure(figsize=(12, 8))
sns.heatmap(
    corr,
    annot=True,
    cmap='coolwarm',
    mask=mask,
    annot_kws={'size': 8},  # Smaller annotations
    fmt=".2f"
)
plt.title(f'Masked Correlation Heatmap (abs(corr) ≥ {threshold})')
plt.show()

### I'm choosing kendall's correlation because it's more suitable for non-linear data, the dataset is relatively small and (as I think) it better shows correlation to the target variable in this specific dataset

# Now let's split data into train & test, while also removing features that do not have high correlation with our target (Price)

In [ ]:
df = df.drop(['basement', 'hotwaterheating'], axis=1)

In [ ]:
#Split into train & test
X = df.drop('price', axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f'Training Features shape: {X_train.shape}, Test Features Shape: {X_test.shape}')

# XGB Model

In [ ]:
xgb = XGBRegressor(random_state=42)
xgb.fit(X_train, y_train)

In [ ]:
#Predictions
y_pred = xgb.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f'Base XGB Model:\nRMSE: {rmse: .2f}\nR2 Score: {r2: .2f}')

In [ ]:
#Fine tune w/ GridSearch
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.6, 0.8, 1.0]
}

print('Optimization initialized.')
grid_search = GridSearchCV(estimator=XGBRegressor(random_state=42), param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, verbose=1, n_jobs=1)
grid_search.fit(X_train, y_train)
print("Here's what we found:", grid_search.best_params_)

In [ ]:
#Model using optimized parameters
better_xgb = grid_search.best_estimator_
y_pred_better = better_xgb.predict(X_test)
rmse_better = np.sqrt(mean_squared_error(y_test, y_pred_better))
r2_better = r2_score(y_test, y_pred_better)

print(f"\nOptimized XGBoost Model:\nRMSE: {rmse_better:.2f}\nR2: {r2_better:.2f}")


# ....

# Visualize

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(y_test, y_pred_better, alpha=0.6, color='orange')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--')
plt.title('Real Price vs. Predicted')
plt.xlabel('Real Price')
plt.ylabel('Predicted Price')
plt.show()

In [ ]:
#Fine tune w/ RandomizedSearchCV
from scipy.stats import randint, uniform  # For parameter distributions

param_dist = {
    'n_estimators': randint(50, 300),  # Random integers in range
    'learning_rate': uniform(0.01, 0.3),  # Random floats in range
    'max_depth': randint(3, 10),  # 3 to 9
    'subsample': uniform(0.5, 0.5)  # 0.5 to 1.0
}

# Initialize RandomizedSearch
random_search = RandomizedSearchCV(
    estimator=XGBRegressor(random_state=42),
    param_distributions=param_dist,
    n_iter=50,  # Number of random combinations to try
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    n_jobs=-1,  # Use all CPU cores
    random_state=42  # For reproducibility
)

random_search.fit(X_train, y_train)
print("Best params:", random_search.best_params_)

In [ ]:
#Model using optimized parameters
rand_better_xgb = random_search.best_estimator_
y_pred_rand_better = better_xgb.predict(X_test)
rand_rmse_better = np.sqrt(mean_squared_error(y_test, y_pred_rand_better))
rand_r2_better = r2_score(y_test, y_pred_rand_better)

print(f"\nOptimized XGBoost Model:\nRMSE: {rand_rmse_better:.2f}\nR2: {rand_r2_better:.2f}")

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(y_test, y_pred_rand_better, alpha=0.6, color='orange')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--')
plt.title('Real Price vs. Predicted')
plt.xlabel('Real Price')
plt.ylabel('Predicted Price')
plt.show()

# Let's try optimizing with Optuna

In [ ]:
#Code is the same up to creating X and y variables, we're going to try log transform on 'price' in order to minimize error


#Split into train & test
X_o = df.drop('price', axis=1)
y_o = np.log1p(df['price']) #log of target variable

X_train_o, X_test_o, y_train_o, y_test_o = train_test_split(X_o, y_o, test_size=0.2, random_state=42)
print(f'Training Features shape: {X_train.shape}, Test Features Shape: {X_test_o.shape}')

In [ ]:
import optuna
#Optuna optimization
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10)
    }
    
    model = XGBRegressor(**params, random_state=42)
    model.fit(X_train_o, y_train_o)
    predictions = model.predict(X_test_o)
    rmse = np.sqrt(mean_squared_error(y_test_o, predictions))
    return rmse

print('Hacking the mainframe...')
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)


best_params = study.best_params
print('Best parameters:', best_params)

In [ ]:
final_model = XGBRegressor(**best_params, random_state=42)
final_model.fit(X_train_o, y_train_o)
final_predictions = final_model.predict(X_test_o)

#Model's rmse
rmse_final = np.sqrt(mean_squared_error(y_test_o, final_predictions))
r2_final = r2_score(y_test_o, final_predictions)
print(f'RMSE After Optuna: {rmse_final:.2f}, R2: {r2_final:.2f}')

In [ ]:
#Visualize results
plt.figure(figsize=(12,8))
plt.scatter(np.expm1(y_test_o), np.expm1(final_predictions), alpha=0.6, color='orange')
plt.plot([np.expm1(y_test_o).min(), np.expm1(y_test_o).max()],
         [np.expm1(y_test_o).min(), np.expm1(y_test_o).max()], '--r')
plt.title('Real Price vs. Predicted')
plt.xlabel('Real Price')
plt.ylabel('Predicted Price')
plt.show()

In [ ]:
#transform data back
final_rmse = np.sqrt(mean_squared_error(np.expm1(y_test_o), np.expm1(final_predictions)))
print(f'RMSE on real data: {final_rmse:.2f}')

In [ ]:
error_analysis = pd.DataFrame({'Real Price': np.expm1(y_test_o), 'Predicted Price': np.expm1(final_predictions)})
error_analysis['Error'] = error_analysis['Real Price'] - error_analysis['Predicted Price']
sns.histplot(error_analysis['Error'], kde=True)
plt.title("Predictions Error")
plt.show()

In [ ]:
#Optuna using Mean absolute error
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10)
    }
    
    model = XGBRegressor(**params, random_state=42)
    model.fit(X_train_o, y_train_o)
    predictions = model.predict(X_test_o)
    mae = mean_absolute_error(y_test_o, predictions)
    return mae

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

final_mae = mean_absolute_error(np.expm1(y_test_o), np.expm1(final_predictions))
print(f"MAE after Optuna: {final_mae:.2f}")

In [ ]:
errors = np.abs(np.expm1(y_test_o) - np.expm1(final_predictions))

#Error histogram
plt.figure(figsize=(10, 6))
sns.histplot(errors, bins=30, kde=True, color='orange')
plt.title("Distribution of absolute errors")
plt.xlabel("Absolute error (price)")
plt.ylabel("Frequency")
plt.show()